<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/TanksInSeriesLT_vs_NonLinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [6]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root

In [5]:
Ac = 1.
Ah = 0.2
g = 9.8

def rhs(t, h, qin):
    dhdt  = (qin - Ah*jnp.sqrt(2*g*h))/Ac
    return dhdt

In [ ]:
dx/dt = A * x + B*u

In [11]:
qin0 = 1.
h0 = root(lambda h: rhs(0.,h, qin0), 0.).x

In [16]:
A=jax.jacobian(rhs, 1)(0., h0, qin0)

In [18]:
B=jax.jacobian(rhs, 2)(0., h0, qin0)